Model Trainning

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import brevitas.nn as qnn

# Define the transformation for dataset (if needed)
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Load the training and test datasets
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Create data loaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.quant_inp1 = qnn.QuantIdentity(bit_width=8, signed=False, return_quant_tensor=True)
        self.fc1 = qnn.QuantLinear(784, 64, bias=False,
                                   weight_bit_width=8,
                                   bias_bit_width=8)
        self.relu1 = qnn.QuantReLU(bit_width=26, return_quant_tensor=True)
        self.quant_inp2 = qnn.QuantIdentity(bit_width=8, signed=False, return_quant_tensor=True)
        self.fc2 = qnn.QuantLinear(64, 64, bias=False,
                                   weight_bit_width=8,
                                   bias_bit_width=8)
        self.relu2 = qnn.QuantReLU(bit_width=22, return_quant_tensor=True)
        self.quant_inp3 = qnn.QuantIdentity(bit_width=8, signed=False, return_quant_tensor=True)
        self.fc3 = qnn.QuantLinear(64, 64, bias=False,
                                   weight_bit_width=8,
                                   bias_bit_width=8)
        self.relu3 = qnn.QuantReLU(bit_width=22, return_quant_tensor=True)
        self.quant_inp4 = qnn.QuantIdentity(bit_width=8, signed=False, return_quant_tensor=True)
        self.fc4 = qnn.QuantLinear(64, 10, bias=False,
                                   weight_bit_width=8,
                                   bias_bit_width=8)
        self.relu4 = qnn.QuantReLU(bit_width=22, return_quant_tensor=True)
        self.quant_inp5 = qnn.QuantIdentity(bit_width=8, signed=False, return_quant_tensor=True)
        
    def forward(self, x):
        x = torch.flatten(x, 1)  # Flatten the input tensor
        x = self.quant_inp1(x)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.quant_inp2(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.quant_inp3(x)
        x = self.fc3(x)
        x = self.relu3(x)
        x = self.quant_inp4(x)
        x = self.fc4(x)
        x = self.relu4(x)
        x = self.quant_inp5(x)
        return x    
# Initialize the model, loss function, and optimizer
model = SimpleNN()

# Define the criterion and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    for batch_idx, (data, targets) in enumerate(train_loader):
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(data)  # Forward pass
        loss = criterion(outputs, targets)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights
        running_loss += loss.item()  # Accumulate loss

    # Print average loss for the epoch
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")

Epoch 1/10, Loss: 1.1844730279974338
Epoch 2/10, Loss: 1.0443896804410002
Epoch 3/10, Loss: 1.0076571052897967
Epoch 4/10, Loss: 0.9869441156829598
Epoch 5/10, Loss: 0.9734681997217858
Epoch 6/10, Loss: 0.962691763062467
Epoch 7/10, Loss: 0.9536674909754348
Epoch 8/10, Loss: 0.9481916370740069
Epoch 9/10, Loss: 0.9414239671311653
Epoch 10/10, Loss: 0.9403496995917769


In [11]:
# Evaluate the model
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0
with torch.no_grad():
    for data, targets in test_loader:
        outputs = model(data)
        _, predicted = torch.max(outputs.value, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

print(f"Accuracy on test set: {100 * correct / total}%")

Accuracy on test set: 59.49%


In [12]:
torch.save(model.state_dict(), 'model_weights_quantized.pth')

# FP32 Trainning

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define the transformation for the dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Load the training and test datasets
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Create data loaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define the SimpleNN model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(784, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 10)
        
    def forward(self, x):
        x = torch.flatten(x, 1)  # Flatten the input tensor
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

# Initialize the model, loss function and optimizer
model_fp32 = SimpleNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_fp32.parameters(), lr=0.001)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    model_fp32.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model_fp32(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

# Save the trained model weights
torch.save(model_fp32.state_dict(), 'model_fp32.pth')


Epoch [1/5], Loss: 0.4320
Epoch [2/5], Loss: 0.2027
Epoch [3/5], Loss: 0.1545
Epoch [4/5], Loss: 0.1295
Epoch [5/5], Loss: 0.1135


In [2]:
import torch
import brevitas.nn as qnn

# Define the SimpleNN model with Brevitas quantization
class SimpleNNQuant(nn.Module):
    def __init__(self):
        super(SimpleNNQuant, self).__init__()
        self.quant_inp1 = qnn.QuantIdentity(bit_width=8, signed=False, return_quant_tensor=True)
        self.fc1 = qnn.QuantLinear(784, 64, bias=False, weight_bit_width=8)
        self.relu1 = qnn.QuantReLU(bit_width=26, return_quant_tensor=True)
        self.quant_inp2 = qnn.QuantIdentity(bit_width=8, signed=False, return_quant_tensor=True)
        self.fc2 = qnn.QuantLinear(64, 64, bias=False, weight_bit_width=8)
        self.relu2 = qnn.QuantReLU(bit_width=22, return_quant_tensor=True)
        self.quant_inp3 = qnn.QuantIdentity(bit_width=8, signed=False, return_quant_tensor=True)
        self.fc3 = qnn.QuantLinear(64, 64, bias=False, weight_bit_width=8)
        self.relu3 = qnn.QuantReLU(bit_width=22, return_quant_tensor=True)
        self.quant_inp4 = qnn.QuantIdentity(bit_width=8, signed=False, return_quant_tensor=True)
        self.fc4 = qnn.QuantLinear(64, 10, bias=False, weight_bit_width=8)
        self.relu4 = qnn.QuantReLU(bit_width=22, return_quant_tensor=True)
        
    def forward(self, x):
        x = torch.flatten(x, 1)  # Flatten the input tensor
        x = self.quant_inp1(x)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.quant_inp2(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.quant_inp3(x)
        x = self.fc3(x)
        x = self.relu3(x)
        x = self.quant_inp4(x)
        x = self.fc4(x)
        x = self.relu4(x)
        return x

# Load the FP32 model weights
model_fp32_weights = torch.load('model_fp32.pth')

# Initialize the quantized model
model_quant = SimpleNNQuant()

# Copy the weights from FP32 model to quantized model
model_quant.fc1.weight.data = model_fp32_weights['fc1.weight'].clone().detach()
model_quant.fc2.weight.data = model_fp32_weights['fc2.weight'].clone().detach()
model_quant.fc3.weight.data = model_fp32_weights['fc3.weight'].clone().detach()
model_quant.fc4.weight.data = model_fp32_weights['fc4.weight'].clone().detach()

# Since bias is not used in quantized model, we do not copy biases

# The quantized model is now ready for further use


No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
